In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings
import gradio 
from datasets import load_dataset
from trl import SFTTrainer

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
 print('메모리 사용량/총 메모리: ' + str(torch.cuda.memory_allocated('cuda:0')/1024**3 ) + 'GB / ' + str(torch.cuda.memory_reserved('cuda:0')/1024**3 ) + 'GB')

메모리 사용량/총 메모리: 0.0GB / 0.0GB


In [4]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: NVIDIA RTX A6000
Compute Capability: (8, 6)
Total Memory: 51041271808 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.2.0-34-generic
Python Version: 3.9.12


In [5]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("lcw99/llama2-ko-chang-13b-instruct-chat")
# model = AutoModelForCausalLM.from_pretrained("lcw99/llama2-ko-chang-13b-instruct-chat")

# Pre trained model
model_name = "Minirecord/Mini_synatra_7b_02" 

# Dataset name
dataset_name = "dohun0714/chat_bot"

# Hugging face repository link to save fine-tuned model(Create new repository in huggingface,copy and paste here)
new_model = "dohun0714/synatra_ko_chat"

In [6]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ········
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through g

In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
# dataset["text"][0]
print(dataset)

Dataset({
    features: ['Q', 'A', 'label'],
    num_rows: 11823
})


In [8]:
# Load base model(llama-2-7b-hf) and tokenizer
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.float16,
#     bnb_4bit_use_double_quant= False,
# )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
#     quantization_config=bnb_config,
#     device_map={"": 0}
)
model.to('cuda:0')
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# for i, data in enumerate(dataset):
#     print(data)
#     print("\n")
#     if i > 10:  # 첫 10개 요소만 출력
#         break

In [10]:
# 예시 코드
example_sentence = "이것은 테스트 문장입니다."
tokenized_output = tokenizer.tokenize(example_sentence)
print(tokenized_output)

# 토크나이저와 모델의 임베딩 레이어 크기 비교
print(len(tokenizer.vocab))  # 토크나이저 단어장 크기
print(model.model.embed_tokens.weight.size(0)) # 모델 임베딩 레이어 크기

['▁', '이', '것', '은', '▁', '테', '스', '트', '▁', '문', '장', '입', '니', '다', '.']
32002
32002


In [11]:
#monitering login
wandb.login(key="ba603a5612d073d6bd76b4b1844ee94b3e05fbbd")
run = wandb.init(project='synatra-kor-chatbot', job_type="training", anonymous="allow")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: dohun562. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/dhkim/.netrc
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disa

In [12]:
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [13]:
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb",
)

In [14]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [15]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['Q'])):
        question = example['Q'][i]
        answer = example['A'][i]
        text = f"### 질문: {question}\n### 답: {answer}"
        output_texts.append(text)
    return output_texts
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    peft_config=peft_config,
    max_seq_length= 50,
#     dataset_text_field="Q",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
) 

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [16]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
30,2.326500
60,1.646300
90,1.378100
120,1.383200
150,1.272400
180,1.317600
210,1.237400
240,1.173300
270,1.216200
300,1.125500


TrainOutput(global_step=739, training_loss=1.2292175163919452, metrics={'train_runtime': 1021.9945, 'train_samples_per_second': 11.569, 'train_steps_per_second': 0.723, 'total_flos': 2.202462026993664e+16, 'train_loss': 1.2292175163919452, 'epoch': 1.0})

In [36]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

NameError: name 'trainer' is not defined

In [18]:
# def stream(user_prompt):
#     runtimeFlag = "cuda:0"
#     system_prompt = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
#     B_INST, E_INST = "### Instruction:\n", "### Response:\n"

#     prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n\n{E_INST}"

#     inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

#     streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

#     # Despite returning the usual output, the streamer will also print the generated text to stdout.
#     _ = model.generate(**inputs, streamer=streamer, max_new_tokens=1500)

In [29]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1000, temperature=1.0, length_penalty=0.2, do_sample=True, num_beams=10)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))

stream("나 오늘 기뻐")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 기뻐### Response: 저도 기뻐요.</s>


In [30]:
stream("나 오늘 혼났어")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 혼났어### Response: 혼란스러운 일이 있었나봐요.</s>


In [31]:
stream("안녕 너는 누구야?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:안녕 너는 누구야?### Response: 저는 인공지능 어시스턴트입니다. 도움이 되고 싶어요. 무엇을 도와드릴까요?</s>


In [33]:
stream("오늘은 슬프네")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:오늘은 슬프네### Response: 슬픈 일이 있었나봐요.</s>


In [ ]:
ajfklajkafakfa;

In [34]:
# Clear the memory footprint
# del model, trainer
# torch.cuda.empty_cache()

In [37]:
model.config.save_pretrained(new_model)
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)